In [1]:
# import sys
# !{sys.executable} -m pip install ing_theme_matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
# import qbstyles
# from ing_theme_matplotlib import mpl_style
import glacierml as gl
from tqdm import tqdm

# Step 1: Organize the data

## A. Load and organize the Glathida dataset

In [2]:
# raw_dataset = pd.read_csv('/data/fast0/datasets/glathida-3.1.0/data/T.csv')
T = pd.read_csv('/home/sa42/data/glac/T_models/T.csv')
T.rename(columns = {
    'LAT':'T_LAT_CENT',
    'LON':'T_LON_CENT',
    'AREA':'T_AREA',
    'MEAN_SLOPE':'T_MEAN_SLOPE',
    'MEAN_THICKNESS':'T_MEAN_THICKNESS',
    'MAXIMUM_THICKNESS':'T_MAXIMUM_THICKNESS' 
},inplace=True)
TT = pd.read_csv('/home/sa42/data/glac/T_models/TT.csv')
TT.rename(columns = {
    'AREA':'TT_AREA',
    'MEAN_SLOPE':'TT_MEAN_SLOPE',
    'MEAN_THICKNESS':'TT_MEAN_THICKNESS',
    'MAXIMUM_THICKNESS':'TT_MAXIMUM_THICKNESS',   
},inplace=True)
TTT = pd.read_csv('/home/sa42/data/glac/T_models/TTT.csv')
TTT.rename(columns = {
    'ELEVATION':'POINT_ELEVATION',
    'THICKNESS':'POINT_THICKNESS',
},inplace=True)
df1 = pd.merge(T,TT, how = 'inner', on='GlaThiDa_ID')
glathida = pd.merge(df1,TTT, how = 'inner', on='GlaThiDa_ID')
glathida = glathida[[
#     'GlaThiDa_ID',
#     'GLACIER_DB',
#     'GLACIER_ID',
    'T_LAT_CENT',
    'T_LON_CENT',
    'T_AREA',
    'T_MEAN_SLOPE',
    'T_MEAN_THICKNESS',
    'T_MAXIMUM_THICKNESS',
#     'LOWER_BOUND',
#     'UPPER_BOUND',
    'TT_AREA',
    'TT_MEAN_SLOPE',
    'TT_MEAN_THICKNESS',
    'TT_MAXIMUM_THICKNESS',
    'POINT_LAT',
    'POINT_LON',
    'POINT_ELEVATION',
    'POINT_THICKNESS',   
]]
glathida = glathida.dropna()

/home/sa42/miniconda3/envs/python-cartopy-f/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (2,4,5,12) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


### separate out features - what will be trained to predict desired attribute

In [3]:
train_dataset = glathida.sample(frac=0.8, random_state=0)
test_dataset = glathida.drop(train_dataset.index)

train_features = train_dataset.copy()
test_features = test_dataset.copy()

#define label - attribute training to be picked
train_labels = train_features.pop("POINT_THICKNESS")
test_labels = test_features.pop("POINT_THICKNESS")

train_features.describe().transpose()[['mean', 'std']]

,mean,std
T_LAT_CENT,45.248784,7.555996
T_LON_CENT,29.965947,36.370518
T_AREA,1.022082,1.387955
T_MEAN_SLOPE,21.421017,5.425289
T_MEAN_THICKNESS,22.792425,16.785250
T_MAXIMUM_THICKNESS,74.305944,45.827023
TT_AREA,0.423253,0.636670
TT_MEAN_SLOPE,23.084023,9.011630
TT_MEAN_THICKNESS,18.169623,17.729830
TT_MAXIMUM_THICKNESS,41.303255,38.766250


# Step 2: Setup the Model

### Establish normalization layers

In [4]:
normalizer = {}
variable_list = list(train_features)
for variable_name in tqdm(variable_list):

    normalizer[variable_name] = gl.preprocessing.Normalization(input_shape=[1,], axis=None)
    normalizer[variable_name].adapt(np.array(train_features[variable_name]))
    
    
normalizer['ALL'] = gl.preprocessing.Normalization(axis=-1)
normalizer['ALL'].adapt(np.array(train_features))

  0%|          | 0/13 [00:00<?, ?it/s]2022-03-19 22:53:20.226564: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-03-19 22:53:20.226611: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (sermeq.ess.washington.edu): /proc/driver/nvidia/version does not exist
2022-03-19 22:53:20.227200: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
100%|██████████| 13/13 [00:15<00:00,  1.21s/it]


In [5]:
linear_model = {}
linear_history = {}
linear_results = {}

# Linear regression

In [6]:
%%time
for variable_name in tqdm(variable_list):

    linear_model[variable_name] = gl.build_linear_model(normalizer[variable_name])
    linear_history[variable_name] = linear_model[variable_name].fit(
                                        train_features[variable_name], train_labels,        
                                        epochs=1,
                                        verbose=0,
                                        validation_split = 0.2)
    
    
    linear_results[variable_name] = linear_model[variable_name].evaluate(
                                        test_features[variable_name],
                                        test_labels, verbose=0)

100%|██████████| 13/13 [00:28<00:00,  2.22s/it]

CPU times: user 36 s, sys: 2.43 s, total: 38.4 s
Wall time: 28.9 s


In [7]:
# fig,ax=plt.subplots(2,6,figsize=(10,10))
# for i, variable_name in enumerate(variable_list):
# #     mpl_style("dark")
#     ax = plt.subplot(4,4,i+1)
#     plt.subplot(4,4,i+1)
#     gl.plot_loss(linear_history[variable_name])
#     ax.set_title(variable_name)
# #     plt.savefig("/home/sa42/notebooks/glac/figs/GTP1_loss.eps")

# Linear regression with multiple inputs

In [8]:
linear_model = gl.build_linear_model(normalizer['ALL'])

linear_history['MULTI'] = linear_model.fit(
train_features, train_labels,        
   epochs=1,
   verbose=0,
   validation_split = 0.2)

linear_results['MULTI'] = linear_model.evaluate(
    test_features,
    test_labels, verbose=0)

In [9]:
# gl.plot_loss(history_full)
# # plt.savefig("/home/sa42/notebooks/glac/figs/GTP1_full_loss.eps")

# DNN regression

## Define regression functions

In [10]:
# test_results = {}
dnn_model = {}
dnn_history = {}
dnn_results = {}

In [11]:
for variable_name in tqdm(variable_list):

    dnn_model[variable_name] = gl.build_dnn_model(normalizer[variable_name])
    dnn_history[variable_name] = dnn_model[variable_name].fit(
                                        train_features[variable_name], train_labels,        
                                        epochs=1,
                                        verbose=0,
                                        validation_split = 0.2)
    dnn_results[variable_name] = dnn_model[variable_name].evaluate(
                                        test_features[variable_name],
                                        test_labels, verbose=0)

100%|██████████| 13/13 [00:52<00:00,  4.01s/it]


In [12]:
# fig,ax=plt.subplots(4,4,figsize=(10,10))
# for i, variable_name in enumerate(variable_list):
#     plt.subplot(4,4,i+1)
#     xmax = np.max(train_features[variable_name])
#     xmin = np.min(train_features[variable_name])
#     x = tf.linspace(xmin, xmax, 101)
#     y = dnn_model[variable_name].predict(x)
#     plot_single_model_variable(x,y,variable_name)

In [13]:
fig,ax=plt.subplots(4,4,figsize=(10,10))
for i, variable_name in enumerate(variable_list):
    ax = plt.subplot(4,4,i+1)
    gl.plot_loss(dnn_history[variable_name])
    ax.set_title(variable_name)
#     plt.savefig("/home/sa42/notebooks/glac/figs/GTP1_dnn_loss.eps")

# Full model

In [14]:
dnn_full_model = gl.build_dnn_model(normalizer['ALL'])

In [20]:
%%time
dnn_history['MULTI'] = dnn_full_model.fit(
    train_features, train_labels,
    validation_split=0.2,
    verbose=0, epochs=1)

dnn_results['MULTI'] = dnn_full_model.evaluate(
    test_features,
    test_labels, verbose=0)

CPU times: user 36.7 s, sys: 7.2 s, total: 43.9 s
Wall time: 3.85 s


<function glacierml.plot_loss(history)>

In [21]:
gl.plot_loss(dnn_history['MULTI'])
# plt.savefig("/home/sa42/notebooks/glac/figs/GTP1_dnn_full_loss.png")

(0.0, 200.0)

In [22]:
y = dnn_full_model.predict(test_features)
plt.plot(test_labels,y,'.')
plt.plot((0,200),(0,200),'-')
plt.xlabel('True Thickness (m)')
plt.ylabel('Model Thickness (m)')
plt.xlim((0,200))
plt.ylim((0,200))
# plt.savefig("/home/sa42/notebooks/glac/figs/GTP1_res.EPS")

# Collect Results

In [23]:
dfs = pd.DataFrame()
for variable_name in list(dnn_history):    
    df1 = pd.DataFrame(dnn_history[variable_name].history)
    df2 = pd.DataFrame(linear_history[variable_name].history)
    df1 = df1.loc[[df1.last_valid_index()]]
    df2 = df2.loc[[df2.last_valid_index()]]
    df1['Architecture'] = 'DNN'
    df2['Architecture'] = 'Linear'
    df1.insert(0, 'Variable', [variable_name])
    df2.insert(0, 'Variable', [variable_name])
    df = pd.concat([df1,df2])
    dfs = dfs.append(df)
df = dfs[[
    'Architecture',
    'Variable',
    'loss',
    'val_loss'
]]
df = df.sort_values(by=['Architecture','Variable'], ascending=[False,False])
print(df.to_latex(index=False))

\begin{tabular}{llrr}
\toprule
Architecture &             Variable &      loss &  val\_loss \\
\midrule
      Linear &     T\_MEAN\_THICKNESS & 18.337793 & 17.566830 \\
      Linear &         T\_MEAN\_SLOPE & 22.358496 & 21.838877 \\
      Linear &  T\_MAXIMUM\_THICKNESS & 16.531616 & 15.452958 \\
      Linear &           T\_LON\_CENT & 20.202122 & 19.124033 \\
      Linear &           T\_LAT\_CENT & 24.117083 & 23.657484 \\
      Linear &               T\_AREA & 16.540230 & 15.481031 \\
      Linear &    TT\_MEAN\_THICKNESS & 21.351662 & 20.732893 \\
      Linear &        TT\_MEAN\_SLOPE & 23.030338 & 22.428825 \\
      Linear & TT\_MAXIMUM\_THICKNESS & 20.424492 & 19.764297 \\
      Linear &              TT\_AREA & 24.115835 & 23.652136 \\
      Linear &            POINT\_LON & 20.187712 & 19.129105 \\
      Linear &            POINT\_LAT & 24.129599 & 23.651413 \\
      Linear &      POINT\_ELEVATION & 21.143362 & 20.207649 \\
      Linear &                MULTI & 15.400393 & 14.134

In [24]:
crash()

NameError: name 'crash' is not defined

# RGI merge